In [ ]:
from regions import Regions
import boto3
from astropy.io import fits
from io import BytesIO
import sys
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt
sys.path.append('/home/bekah/gPhoton2')
from gPhoton.coadd import zero_flag_and_edge


In [ ]:

def regions_to_mask(regions, shape):
    mask = np.zeros(shape, dtype=bool)
    for region in regions:
        try:
            if hasattr(region, 'radius') and (not np.isscalar(region.radius) or region.radius <= 0):
                continue
            region_mask = region.to_mask(mode='center').to_image(shape)
            mask |= region_mask.astype(bool)
        except Exception as e:
            print(f"Skipping region due to error: {e}")
    return mask

In [ ]:
# convert regions file from ds9 to binary fits mask 

region_file = '/home/bekah/glcat/masks/drawn/nuv_spots.reg'

regions = Regions.read(region_file, format='ds9')

fits_file = '/home/bekah/glcat/flats/flat_fuv_normalized.fits'

with fits.open(fits_file) as hdul:
    data = hdul[0].data
    header = hdul[0].header

mask = regions_to_mask(regions, data.shape)
mask_hdu = fits.PrimaryHDU(data=mask.astype(int), header=header)
mask_hdu.writeto('nuv_spots.fits', overwrite=True)

In [ ]:
# combine the three masks for FUV (edge, spots, bright edge area) into a bitwise mask 

mask_file_3 = '/home/bekah/glcat/masks/drawn/fuv_drawn_mask.fits'
mask_file_1 = '/home/bekah/glcat/masks/drawn/nuv_edge.fits'
mask_file_2 = '/home/bekah/glcat/masks/drawn/nuv_spots.fits'

with fits.open(mask_file_1) as hdul:
    mask1 = hdul[0].data
    header = hdul[0].header
with fits.open(mask_file_2) as hdul:
    mask2 = hdul[0].data
with fits.open(mask_file_3) as hdul:
    mask3 = hdul[0].data
    
if mask1.shape != mask2.shape or mask1.shape != mask3.shape:
    raise ValueError("All masks must have the same shape")
combined_mask = (mask1.astype(int) << 0) | (mask2.astype(int) << 1)

combined_mask_hdu = fits.PrimaryHDU(data=combined_mask, header=header)
combined_mask_hdu.writeto('/home/bekah/glcat/masks/drawn/nuv_combined_mask.fits', overwrite=True)

In [ ]:
# downscale and crop combo masks 

from scipy.ndimage import zoom
def downscale_fits_image(input_path, output_path, scale_factor=1/5.35):
    with fits.open(input_path) as hdul:
        image_data = hdul[0].data
        crop_amount = 260
        cropped_array = image_data[crop_amount:-crop_amount, crop_amount:-crop_amount]
        
        downscaled_image_data = zoom(cropped_array, zoom=scale_factor, order=3)
        
        # flip array
        downscaled_image_data == np.rot90(np.fliplr(downscaled_image_data))

        hdu = fits.PrimaryHDU(downscaled_image_data)
        hdul_new = fits.HDUList([hdu])
        hdul_new.writeto(output_path, overwrite=True)
    return downscaled_image_data
input_path = f'/home/bekah/glcat/masks/drawn/fuv_combined_mask.fits'
output_path = f'/home/bekah/glcat/masks/drawn/fuv_combined_mask_ds_1.fits'
downscaled_image_data = downscale_fits_image(input_path, output_path)

In [ ]:
#vcombine og mask and new mask for few spots 

# convert regions files from ds9 to binary fits mask 
region_files = ['fuv_oldnew_edge','fuv_oldnew_brightspot',
                'fuv_oldnew_spots','nuv_oldnew_border', 
                'nuv_oldnew_spots']


fits_file = '/home/bekah/glcat/masks/drawn/fuv_combined_mask_ds_1.fits'

with fits.open(fits_file) as hdul:
    data = hdul[0].data
    header = hdul[0].header

for file in region_files: 
    regions = Regions.read(f'/home/bekah/glcat/masks/drawn/{file}.reg')
    mask = regions_to_mask(regions, data.shape)
    mask_hdu = fits.PrimaryHDU(data=mask.astype(int), header=header)
    mask_hdu.writeto(f'/home/bekah/glcat/masks/drawn/{file}.fits', overwrite=True)


# fuv combo mask 
with fits.open('/home/bekah/glcat/masks/drawn/fuv_oldnew_brightspot.fits') as hdul:
    mask2 = hdul[0].data
with fits.open('/home/bekah/glcat/masks/drawn/fuv_oldnew_spots.fits') as hdul:
    mask1 = hdul[0].data
combined_mask = (mask1.astype(int) << 0) | (mask2.astype(int) << 1)

combined_mask_hdu = fits.PrimaryHDU(data=combined_mask, header=header)
combined_mask_hdu.writeto('/home/bekah/glcat/masks/drawn/fuv_combo_mask_final.fits', overwrite=True)
                           
                           
# nuv combo mask 
with fits.open('/home/bekah/glcat/masks/drawn/nuv_oldnew_spots.fits') as hdul:
    mask1 = hdul[0].data
combined_mask = (mask1.astype(int) << 0)

combined_mask_hdu = fits.PrimaryHDU(data=combined_mask, header=header)
combined_mask_hdu.writeto('/home/bekah/glcat/masks/drawn/nuv_combo_mask_final.fits', overwrite=True)

In [ ]:
# convert to binary (0, 1, 1 = masked)

def tobinary(input_fits_file, output_fits_file):
    with fits.open(input_fits_file) as hdul:
        image_data = hdul[0].data
        binary_image_data = (image_data > 0).astype(np.int32)
        hdu = fits.PrimaryHDU(binary_image_data)
        hdu.writeto(output_fits_file, overwrite=True)
    
tobinary('/home/bekah/glcat/masks/drawn/nuv_combo_mask_final.fits', '/home/bekah/glcat/masks/drawn/nuv_combo_mask_final_b.fits')
tobinary('/home/bekah/glcat/masks/drawn/fuv_combo_mask_final.fits', '/home/bekah/glcat/masks/drawn/fuv_combo_mask_final_b.fits')